In [1]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import imodelsx.process_results
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import analyze_helper, viz
# from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')
# from neurosynth import term_dict, term_dict_rev
# import viz
from load_coef_flatmaps import _load_coefs_individual, _load_coefs_full, \
_load_coefs_individual_wordrate, _load_coefs_wordrate, _load_coefs_shapley, _load_coefs_individual_gpt4
# imodelsx.process_results.delete_runs_in_dataframe(
    # rr[rr.use_added_wordrate_feature == 1], actually_delete=True)

In [2]:
# results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug14_neurosynth_gemv'
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/oct17_neurosynth_gemv'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
imodelsx.process_results.delete_runs_in_dataframe(
    rr[rr.use_test_setup == 1], actually_delete=True)
rr.to_pickle('oct17_tmp.pkl')

100%|██████████| 2157/2157 [30:31<00:00,  1.18it/s]
/home/chansingh/imodelsx/imodelsx/process_results.py:92: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[k] = df[k].fillna(np.nan)


experiment varied these params: ['subject', 'pc_components', 'feature_selection_alpha', 'feature_selection_stability_seeds', 'use_added_wordrate_feature', 'qa_embedding_model', 'qa_questions_version', 'use_random_subset_features', 'single_question_idx', 'ndelays', 'seed']


0it [00:00, ?it/s]


Deleted 0/0 directories.


In [4]:
rr_all = pd.read_pickle('oct17_tmp.pkl')

In [2]:
# # this has the most recent result
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug16_gpt4'
rr_gpt4, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
rr_gpt4 = rr_gpt4[rr_gpt4.use_test_setup == 0]
imodelsx.process_results.delete_runs_in_dataframe(
    rr_gpt4[rr_gpt4.use_test_setup == 1], actually_delete=True)
# rr_gpt4.use_added_wordrate_feature.value_counts()

100%|██████████| 172/172 [00:18<00:00,  9.15it/s]
/home/chansingh/imodelsx/imodelsx/process_results.py:92: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[k] = df[k].fillna(np.nan)


experiment varied these params: ['use_added_wordrate_feature', 'qa_questions_version']


0it [00:00, ?it/s]

Deleted 0/0 directories.


In [5]:
def save_df(df, subject, filename):
    if df:
        joblib.dump(df, join(PROCESSED_DIR, subject, filename))
        print(f'saved {subject} {filename}')


# subject = 'S02'
for subject in ['S01', 'S02', 'S03']:
    # for subject in ['S02', 'S01', 'S03']:
    # for subject in [f'S0{i}' for i in range(1, 9)]:
    os.makedirs(join(PROCESSED_DIR, subject), exist_ok=True)

    # print('shapes', rr_8.shape, rr_4.shape)
    for ndelays, suffix1 in zip([4], ['']):
        # for ndelays, suffix1 in zip([8, 4, 1], ['_ndel=8', '', '_ndel=1']):
        rr = rr_all[rr_all.ndelays == ndelays]
        if len(rr) == 0:
            continue

        for pc_components, suffix in zip([-1, 100], ['', '_pc']):
            r = rr[rr.pc_components == pc_components]
            if len(r) == 0:
                continue

            r_gpt = r[r.qa_embedding_model == 'gpt4']
            df = _load_coefs_individual_gpt4(r_gpt, subject=subject)
            save_df(df, subject, f'individual_gpt4{suffix1}{suffix}.pkl')
            df = _load_coefs_full(r_gpt, subject='S02', qa_questions_version='qs_35',
                                  use_added_wordrate_feature=0)
            save_df(df, subject, f'full_35_gpt4{suffix1}{suffix}.pkl')
            df = _load_coefs_shapley(
                r_gpt, subject, qa_questions_version='qs_35')
            save_df(df, subject, f'shapley_35_gpt4{suffix1}{suffix}.pkl')

            # r = r[r.qa_embedding_model == 'ensemble2']
            # df = _load_coefs_full(
            #     r, subject=subject, qa_questions_version='v3_boostexamples_merged')
            # save_df(df, subject, f'full_35{suffix1}{suffix}.pkl')
            # df = _load_coefs_full(
            #     r, subject=subject, qa_questions_version='v3_boostexamples_merged', use_added_wordrate_feature=1)
            # save_df(df, subject, f'full_35_wordrate{suffix1}{suffix}.pkl')
            # df = _load_coefs_full(
            #     r, subject=subject, qa_questions_version='v1neurosynth')
            # save_df(df, subject, f'full_neurosynth{suffix1}{suffix}.pkl')
            # df = _load_coefs_full(
            #     r, subject=subject, qa_questions_version='v1neurosynth', use_added_wordrate_feature=1)
            # save_df(df, subject,
            #         f'full_neurosynth_wordrate{suffix1}{suffix}.pkl')

        # OLD STUFF
        # df = _load_coefs_individual_gpt4(rr_gpt4, subject=subject)
        # joblib.dump(df, join(PROCESSED_DIR, subject, 'individual_gpt4.pkl'))

        # df = _load_coefs_individual_gpt4(
        #     rr_gpt4, subject='S02', use_added_wordrate_feature=1)
        # joblib.dump(df, join(PROCESSED_DIR, subject,
        #             'individual_gpt4_wordrate.pkl'))

        # df = _load_coefs_individual(
        # rr, subject=subject, qa_questions_version='v1neurosynth')
        # joblib.dump(df, join(PROCESSED_DIR, subject, 'individual_neurosynth.pkl'))

        # df = _load_coefs_individual(
        #     rr, subject=subject, qa_questions_version='v3_boostexamples_merged')
        # joblib.dump(df, join(PROCESSED_DIR, subject, 'individual_35.pkl'))

        # df = _load_coefs_shapley(
        #     rr, subject, qa_questions_version='v3_boostexamples_merged')
        # joblib.dump(df, join(
        #     PROCESSED_DIR, subject, 'shapley_35.pkl'))

        # df = _load_coefs_shapley(
        #     rr, subject, qa_questions_version='v1neurosynth')
        # joblib.dump(df, join(PROCESSED_DIR,
        #             subject, 'shapley_neurosynth.pkl'))

        ########### use old models ###################
        # jointly fitted 35-question model
        # df_w_selected35 = _load_coefs_35questions(subject=subject)

        # individually fitted question models
        # df_w_individual = _load_coefs_individual(rr_shapley, subject=subject)
        # joblib.dump(df_w_individual, join(PROCESSED_DIR,
        # subject, 'individual.pkl'))

        # individually fitted question models *with wordrate
        # df_w_individual_wordrate = _load_coefs_individual_wordrate(
        # subject=subject)

        # wordrate
        # df_w_wordrate_alone = _load_coefs_wordrate(subject=subject)

        # # collate individual dfs #########################
        # # average weights for df_w_selected35 and df_w_individual
        # if subject == 'S02':
        #     df_avg = df_w_selected35.merge(df_w_individual, on='question')
        #     df_avg['weights'] = df_avg.apply(
        #         lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

        # df_avg_individual = df_w_individual.merge(
        #     df_w_individual_wordrate, on='question')
        # df_avg_individual['weights'] = df_avg_individual.apply(
        #     lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

        # df_qa_dict = {
        #     'selected35': df_w_selected35,
        #     'individual': df_w_individual,
        #     'individual_wordrate': df_w_individual_wordrate,
        #     'wordrate_alone': df_w_wordrate_alone,
        #     # 'avg': df_avg,
        #     'shapley_neurosynth': df_w_shapley_neurosynth,
        #     'shapley35': df_w_shapley35,
        #     'avg_individual': df_avg_individual
        # }
        # joblib.dump(df_qa_dict, f'df_qa_dict_{subject}.pkl')

0it [00:00, ?it/s]
0it [00:00, ?it/s]

	 35 questions 35 weights
corr 0.09298482340272012
saved S01 full_35_gpt4_pc.pkl


0it [00:00, ?it/s]
100%|██████████| 35/35 [01:43<00:00,  2.96s/it]


saved S02 individual_gpt4_pc.pkl
	 35 questions 35 weights
corr 0.09298482340272012
saved S02 full_35_gpt4_pc.pkl


100%|██████████| 50/50 [02:53<00:00,  3.46s/it]


saved S02 shapley_35_gpt4_pc.pkl


0it [00:00, ?it/s]
0it [00:00, ?it/s]


	 35 questions 35 weights
corr 0.09298482340272012
saved S03 full_35_gpt4_pc.pkl


### export eng1000 model

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug16_eng1000'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
r = rr[rr.feature_space == 'eng1000']
r = r[r.num_stories == -1]
r = r[r.ndelays == 8]
r = r[r.feature_selection_alpha == -1.000000]
for subject in ['S02', 'S01', 'S03']:
    row = r[r.subject == subject]
    assert len(row) == 1
    row = row.iloc[0]
    weights, weights_pc = flatmaps_per_question.get_weights_top(row)
    print(weights.shape)
    joblib.dump(weights, join(PROCESSED_DIR, subject, 'eng1000_weights.pkl'))